In [1]:
! pip3 install --upgrade --quiet --user google-cloud-aiplatform==1.88.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.14.4 requires protobuf<7.0,==6.31.1, but you have protobuf 5.29.5 which is incompatible.
kfp-pipeline-spec 2.14.0 requires protobuf<7.0,==6.31.1, but you have protobuf 5.29.5 which is incompatible.


In [ ]:
# # Restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [2]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-01-352b2bc21dda


In [3]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

# Task 2. Define functions in Python


In [5]:
# TODO: Define a function to add two numerical inputs and return the result.
# Keep the print statement within the function.
# Define a function to add two numerical inputs and return the result.
def add(a: float, b: float):
    """Adds two numbers together."""
    print("Calling add function")
    return a + b
    

# TODO: Define a function to multiply two numerical inputs and return the result.
# Keep the print statement within the function.
# Define a function to multiply two numerical inputs and return the result.
def multiply(a: float, b: float):
    """Multiplies two numbers together."""
    print("Calling multiply function")
    return a * b





# TODO: Create FunctionDeclarations for your functions
add_func = FunctionDeclaration(
    name="add",
    description="Add two numbers",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "The first number"},
            "b": {"type": "number", "description": "The second number"},
        },
        "required": ["a", "b"],
    },
)

multiply_func = FunctionDeclaration(
    name="multiply",
    description="Multiply two numbers",
    parameters={
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "The first number"},
            "b": {"type": "number", "description": "The second number"},
        },
        "required": ["a", "b"],
    },
)



# Define the tool that contains the function declarations
calculator_tool = Tool(
    function_declarations=[add_func, multiply_func],
)

In [7]:
add_func

name: "add"
description: "Add two numbers"
parameters {
  type_: OBJECT
  properties {
    key: "b"
    value {
      type_: NUMBER
      description: "The second number"
    }
  }
  properties {
    key: "a"
    value {
      type_: NUMBER
      description: "The first number"
    }
  }
  required: "a"
  required: "b"
  property_ordering: "a"
  property_ordering: "b"
}

In [8]:
multiply_func

name: "multiply"
description: "Multiply two numbers"
parameters {
  type_: OBJECT
  properties {
    key: "b"
    value {
      type_: NUMBER
      description: "The second number"
    }
  }
  properties {
    key: "a"
    value {
      type_: NUMBER
      description: "The first number"
    }
  }
  required: "a"
  required: "b"
  property_ordering: "a"
  property_ordering: "b"
}

In [9]:
calculator_tool

function_declarations {
  name: "add"
  description: "Add two numbers"
  parameters {
    type_: OBJECT
    properties {
      key: "b"
      value {
        type_: NUMBER
        description: "The second number"
      }
    }
    properties {
      key: "a"
      value {
        type_: NUMBER
        description: "The first number"
      }
    }
    required: "a"
    required: "b"
    property_ordering: "a"
    property_ordering: "b"
  }
}
function_declarations {
  name: "multiply"
  description: "Multiply two numbers"
  parameters {
    type_: OBJECT
    properties {
      key: "b"
      value {
        type_: NUMBER
        description: "The second number"
      }
    }
    properties {
      key: "a"
      value {
        type_: NUMBER
        description: "The first number"
      }
    }
    required: "a"
    required: "b"
    property_ordering: "a"
    property_ordering: "b"
  }
}

In [6]:
# Define and initialize the model
model = GenerativeModel(
    "gemini-2.0-flash-001",
    generation_config=GenerationConfig(temperature=0),
    tools=[calculator_tool],
    system_instruction="""- Fulfill the user's instructions, including telling jokes.
- Answer the user's question using the appropriate function tool if available.
- You may call both the 'multiply' and 'add' functions one after the other if needed.
- Use the 'multiply' function only for multiplication-related tasks.
- Use the 'add' function only for addition-related tasks.
- A function tool may only be invoked if its capabilities are an exact match for the described task. Avoid speculative or improper use of tools.
- In the absence of a suitable tool, respond to the user with a generic answer without performing the mathematical calculation yourself."""
)

/opt/conda/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [10]:
chat = model.start_chat()

In [ ]:
# def handle_response(response):

#     # If there is a function call then invoke it
#     # Otherwise print the response.
#     if response.candidates[0].function_calls:
#         function_call = response.candidates[0].function_calls[0]
#     else:
#         print(response.text)
#         return

#     # TODO: Complete the following sections
#     if # the function_call requests your add function
#         # Extract the arguments to use in your function
#         # Call your function
#         # Send the result back to the chat session with the model
#         # Make a recursive call of this handler function

#     elif # the function_call requests your multiply function
#         # Extract the arguments to use in your function
#         # Call your function
#         # Send the result back to the chat session with the model
#         # Make a recursive call of this handler function

#     else:
#         # You shouldn't end up here
#         print(function_call)

In [11]:
def handle_response(response):
    """Handles the response from the model, invoking functions if necessary."""
    if response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call

        if function_call.name == "add":
            a = function_call.args["a"]
            b = function_call.args["b"]
            result = add(a, b)
            response = chat.send_message(
                Part.from_function_response(
                    name="add",
                    response={
                        "result": result,
                    },
                ),
            )
            handle_response(response)

        elif function_call.name == "multiply":
            a = function_call.args["a"]
            b = function_call.args["b"]
            result = multiply(a, b)
            response = chat.send_message(
                Part.from_function_response(
                    name="multiply",
                    response={
                        "result": result,
                    },
                ),
            )
            handle_response(response)
        else:
            print(f"Unknown function call: {function_call.name}")

    else:
        print(response.candidates[0].content.parts[0].text)

In [12]:
# Example usage:
print("Sending message: What is 23 * 7?")
response = chat.send_message("What is 23 * 7?")
handle_response(response)


Sending message: What is 23 * 7?
Calling multiply function
The answer is 161.



In [13]:
print("\nSending message: What is 100 + 51?")
response = chat.send_message("What is 100 + 51?")
handle_response(response)



Sending message: What is 100 + 51?
Calling add function
The answer is 151.



In [15]:
print("\nSending message: What is 100 - 51?")
response = chat.send_message("What is 100 - 51?")
handle_response(response)



Sending message: What is 100 - 51?
I am sorry, I cannot perform subtraction. I can only add or multiply.



In [16]:
print("\nSending message: What is 100 / 51?")
response = chat.send_message("What is 100 / 51?")
handle_response(response)



Sending message: What is 100 / 51?
I am sorry, I cannot perform division. I can only add or multiply.



In [17]:
print("\nSending message: What is 100 % 51?")
response = chat.send_message("What is 100 % 51?")
handle_response(response)



Sending message: What is 100 % 51?
I am sorry, I cannot perform modulo operations. I can only add or multiply.



In [18]:
print("\nSending message: What is 10 ** 2?")
response = chat.send_message("What is 100 ** 2?")
handle_response(response)



Sending message: What is 10 ** 2?
I am sorry, I cannot perform exponentiation. I can only add or multiply.



In [14]:
print("\nSending message: Tell me a joke.")
response = chat.send_message("Tell me a joke.")
handle_response(response)


Sending message: Tell me a joke.
Why don't scientists trust atoms?

Because they make up everything!



# Task 3. Prompt the model to call your functions


In [19]:
response = chat.send_message("Tell me a joke?")
handle_response(response)

Why did the scarecrow win an award? Because he was outstanding in his field!



In [20]:
response = chat.send_message("I have 7 pizzas each with 16 slices. How many slices do I have?")
handle_response(response)

Calling multiply function
You have 112 slices.



In [21]:
# sanity check
7*16

112

In [22]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. How many pizzas did they bring together?")
handle_response(response)

Calling add function
They brought 7 pizzas together.



In [23]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. There are 16 slices per pizza. How many slices are there?")
handle_response(response)    

Calling add function
Calling multiply function
There are 112 slices.



In [24]:
response = chat.send_message("Doug brought 4 pizzas, but Andrew dropped 2 on the ground. How many pizzas are left?")
handle_response(response)

I am sorry, I cannot perform subtraction. I can only add or multiply.

